In [1]:
import polars as pl
import gc
import pickle
from pathlib import Path, PosixPath
from tqdm.auto import tqdm

import sys
sys.path.append('..')

from src.utils import seed_everything, get_logger, get_config, TimeUtil
from src.utils.competition_utils import clipping_input
from src.data import DataProvider, FeatureEngineering, Preprocessor, HFPreprocessor
from src.train import get_dataloader

In [2]:
# コマンドライン引数
exp = '146'

In [3]:
config = get_config(exp, config_dir=Path('../config'))
logger = get_logger(config.output_path)
logger.info(f'exp: {exp} | run_mode={config.run_mode}, multi_task={config.multi_task}, loss_type={config.loss_type}')

seed_everything(config.seed)

[ 2024-10-10 14:50:56 | INFO ] exp: 146 | run_mode=hf, multi_task=False, loss_type=mae


In [4]:
config.run_mode = 'dev'
config.multi_task = False

In [5]:
with TimeUtil.timer('Data Loading...'):
    dpr = DataProvider(config)
    train_df, test_df = dpr.load_data()

[Data Loading...] start [0.5GB(4.2%)]


In [6]:
with TimeUtil.timer('Feature Engineering...'):
    fer = FeatureEngineering(config)
    train_df = fer.feature_engineering(train_df)
    test_df = fer.feature_engineering(test_df)

[Feature Engineering...] start [66.5GB(38.1%)]
[Feature Engineering...] done [69.4GB(30.2%)(+2.975GB)] 27.8554 s


In [7]:
with TimeUtil.timer('Scaling and Clipping Features...'):
    ppr = Preprocessor(config)
    train_df, test_df = ppr.scaling(train_df, test_df)
    input_cols, target_cols = ppr.input_cols, ppr.target_cols
    if config.task_type == 'grid_pred':
        train_df = train_df.drop(target_cols)

    valid_df = train_df.filter(pl.col('fold') == 0)
    train_df = train_df.filter(pl.col('fold') != 0)
    valid_df, input_clip_dict = clipping_input(train_df, valid_df, input_cols)
    test_df, _ = clipping_input(None, test_df, input_cols, input_clip_dict)
    pickle.dump(input_clip_dict, open(config.output_path / 'input_clip_dict.pkl', 'wb'))

[Scaling and Clipping Features...] start [69.4GB(30.2%)]
[Scaling and Clipping Features...] done [34.4GB(22.5%)(-35.001GB)] 36.5791 s


In [8]:
with TimeUtil.timer('Converting to arrays for NN...'):
    array_data = ppr.convert_numpy_array(train_df, valid_df, test_df)
    del train_df, valid_df, test_df
    gc.collect()

[Converting to arrays for NN...] start [34.4GB(22.5%)]
[Converting to arrays for NN...] done [53.5GB(40.7%)(+19.076GB)] 179.8489 s


In [9]:
# Prepare HF Data
if config.run_mode == 'hf':
    with TimeUtil.timer('HF Data Preprocessing...'):
        hf_ppr = HFPreprocessor(config)
        # hf_pcr.preprocess_data()
        # hf_pcr.convert_numpy_array(near_target=False)
        # del train_loader; gc.collect()
        # train_loader = None

In [10]:
with TimeUtil.timer('Creating Torch DataLoader...'):
    train_loader = get_dataloader(
        config,
        array_data['train_ids'],
        array_data['X_train'],
        array_data['y_train'],
        is_train=True
    )
    valid_loader = get_dataloader(
        config,
        array_data['valid_ids'],
        array_data['X_valid'],
        array_data['y_valid'],
        is_train=False
    )
    test_loader = get_dataloader(
        config,
        array_data['test_ids'],
        array_data['X_test'],
        is_train=False
    )
    del array_data
    gc.collect()

[Creating Torch DataLoader...] start [53.5GB(40.7%)]
[Creating Torch DataLoader...] done [53.5GB(40.6%)(+0.000GB)] 0.0876 s


# Trainer

In [31]:
import pickle
from collections import defaultdict
from typing import Dict, List, Literal, Tuple

import loguru
import numpy as np
import polars as pl
import torch
from omegaconf import DictConfig, OmegaConf
from torch import nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from src.train import ComponentFactory
from src.train.train_utils import AverageMeter
from src.utils import clean_message
from src.utils.competition_utils import evaluate_metric, get_io_columns, get_sub_factor
from src.utils.constant import (
    PP_TARGET_COLS,
    SCALER_TARGET_COLS,
    TARGET_MIN_MAX,
    VERTICAL_TARGET_COLS,
)


class Trainer:
    def __init__(self, config: DictConfig, logger: loguru._Logger, save_suffix: str = ""):
        self.config = config
        self.eval_step = config.eval_step[config.run_mode]
        self.logger = logger
        self.save_suffix = save_suffix
        self.detail_pbar = True

        self.model = ComponentFactory.get_model(config)
        self.model = self.model.to(config.device)
        n_device = torch.cuda.device_count()
        if n_device > 1:
            self.model = nn.DataParallel(self.model)
        self.loss_fn = ComponentFactory.get_loss(config)
        self.train_loss = AverageMeter()
        self.valid_loss = AverageMeter()

        _, self.target_cols = get_io_columns(config)
        self.model_target_cols = self.get_model_target_cols()
        self.factor_dict = get_sub_factor(config.input_path, old=False)
        self.old_factor_dict = get_sub_factor(config.input_path, old=True)

        self.y_numerators = np.load(
            config.output_path / f"y_numerators_{config.target_scale_method}.npy"
        )
        self.y_denominators = np.load(
            config.output_path / f"y_denominators_{config.target_scale_method}.npy"
        )
        self.target_min_max = [TARGET_MIN_MAX[col] for col in self.target_cols]

        self.valid_ids = None
        self.test_ids = None
        self.valid_pp_df = None
        self.test_pp_df = None
        self.pp_run = True
        self.pp_y_cols = PP_TARGET_COLS
        self.pp_x_cols = [col.replace("ptend", "state") for col in self.pp_y_cols]

        self.best_score_dict = defaultdict(lambda: (-1, -np.inf))

    def train(
        self,
        train_loader: DataLoader,
        valid_loader: DataLoader,
        colwise_mode: bool = True,
        retrain: bool = False,
        retrain_weight_name: str = "",
        retrain_best_score: float = -np.inf,
        eval_only: bool = False,
    ):
        if eval_only:
            self.best_score_dict = pickle.load(
                open(self.config.output_path / f"best_score_dict{self.save_suffix}.pkl", "rb")
            )
            eval_method = "colwise" if colwise_mode else "single"
            score, cw_score, preds, _ = self.valid_evaluate(
                valid_loader, current_epoch=-1, eval_count=-1, eval_method=eval_method
            )
            self.save_oof_df(self.valid_ids, preds)
            return score, cw_score, -1

        self.optimizer = ComponentFactory.get_optimizer(self.config, self.model)
        self.scheduler = ComponentFactory.get_scheduler(
            self.config, self.optimizer, steps_per_epoch=len(train_loader)
        )
        global_step = 0
        eval_count = 0
        best_score = -np.inf

        if retrain:
            self.best_score_dict = pickle.load(
                open(self.config.output_path / f"best_score_dict{self.save_suffix}.pkl", "rb")
            )
            self.model.load_state_dict(
                torch.load(self.config.output_path / f"{retrain_weight_name}.pth")
            )
            weight_numbers = [
                int(file.stem.split("_")[-1].replace("eval", ""))
                for file in list(self.config.output_path.glob(f"model{self.save_suffix}_eval*.pth"))
            ]
            eval_count = sorted(weight_numbers)[-1] + 1
            best_score = retrain_best_score

        # 学習ループの開始
        for epoch in tqdm(range(self.config.epochs)):
            self.model.train()
            self.train_loss.reset()

            iterations = (
                tqdm(train_loader, total=len(train_loader)) if self.detail_pbar else train_loader
            )
            for data in iterations:
                _, loss = self.forward_step(data, calc_loss=True)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                self.train_loss.update(loss.item(), n=data[0].size(0))
                global_step += 1

                if global_step % self.eval_step == 0:
                    score, _, preds, update_num = self.valid_evaluate(
                        valid_loader,
                        current_epoch=epoch,
                        eval_count=eval_count,
                        eval_method="single",
                    )
                    if colwise_mode and update_num > 0:
                        torch.save(
                            self.model.state_dict(),
                            self.config.output_path
                            / f"model{self.save_suffix}_eval{eval_count}.pth",
                        )

                    if score > best_score:
                        best_score = score
                        best_preds = preds
                        best_epochs = epoch
                        torch.save(
                            self.model.state_dict(),
                            self.config.output_path / f"model{self.save_suffix}_best.pth",
                        )

                    eval_count += 1
                    self.model.train()

            message = f"""
                [Train] :
                    Epoch={epoch},
                    Loss={self.train_loss.avg:.5f},
                    LR={self.optimizer.param_groups[0]["lr"]:.5e}
            """
            self.logger.info(clean_message(message))

        if colwise_mode:
            self.remove_unuse_weights()
            best_score, best_cw_score, best_preds, _ = self.valid_evaluate(
                valid_loader, current_epoch=-1, eval_count=-1, eval_method="colwise"
            )

        self.save_oof_df(self.valid_ids, best_preds)
        return best_score, best_cw_score, best_epochs

    def valid_evaluate(
        self,
        valid_loader: DataLoader,
        current_epoch: int,
        eval_count: int,
        eval_method: Literal["single", "colwise"] = "single",
    ):
        if self.valid_ids is None:
            self.valid_ids = valid_loader.dataset.ids

        if eval_method == "single":
            load_best_weight = True if eval_count == -1 else False
            preds = self.inference_loop(
                valid_loader, mode="valid", load_best_weight=load_best_weight
            )
        elif eval_method == "colwise":
            preds = self.inference_loop_colwise(valid_loader, "valid", self.best_score_dict)

        labels = valid_loader.dataset.y
        if self.config.target_shape == "3dim":
            labels = self.convert_target_3dim_to_2dim(labels)
        preds = self.restore_pred(preds)
        labels = self.restore_pred(labels)

        if self.pp_run and self.valid_pp_df is None:
            self.load_postprocess_input("valid")
        if self.pp_run:
            preds = self.postprocess(preds, run_type="valid")
        if self.config.out_clip:
            preds = self.clipping_pred(preds)

        eval_idx = [
            i for i, col in enumerate(self.target_cols) if self.factor_dict[col] != 0
        ]  # factor_dictの値が0のものは自動でR2=1になるようにする
        score, indiv_scores = evaluate_metric(preds, labels, eval_idx=eval_idx)
        cw_score, update_num = self.update_best_score(indiv_scores, eval_count)

        message = f"""
            [Valid] :
                Epoch={current_epoch},
                Loss={self.valid_loss.avg:.5f},
                Score={score:.5f},
                Best Col-Wise Score={cw_score:.5f}
        """
        self.logger.info(clean_message(message))
        return score, cw_score, preds, update_num

    def test_predict(
        self, test_loader: DataLoader, eval_method: Literal["single", "colwise"] = "single"
    ):
        if self.test_ids is None:
            self.test_ids = test_loader.dataset.ids

        if eval_method == "single":
            preds = self.inference_loop(test_loader, mode="test", load_best_weight=True)
        elif eval_method == "colwise":
            self.best_score_dict = pickle.load(
                open(self.config.output_path / f"best_score_dict{self.save_suffix}.pkl", "rb")
            )
            preds = self.inference_loop_colwise(test_loader, "test", self.best_score_dict)

        preds = self.restore_pred(preds)
        if self.pp_run and self.test_pp_df is None:
            self.load_postprocess_input("test")
        if self.pp_run:
            preds = self.postprocess(preds, run_type="test")
        if self.config.out_clip:
            preds = self.clipping_pred(preds)

        pred_df = pl.DataFrame(preds, schema=self.target_cols)
        pred_df = pred_df.with_columns(sample_id=pl.Series(self.test_ids))
        return pred_df

    def inference_loop(
        self,
        eval_loader: DataLoader,
        mode: Literal["valid", "test"],
        load_best_weight: bool = False,
    ):
        self.model.eval()
        if mode == "valid":
            self.valid_loss.reset()

        # テストデータを推論するときはbest_weightを読み込む
        if load_best_weight:
            self.model.load_state_dict(
                torch.load(self.config.output_path / f"model{self.save_suffix}_best.pth")
            )

        preds = []
        with torch.no_grad():
            iterations = (
                tqdm(eval_loader, total=len(eval_loader)) if self.detail_pbar else eval_loader
            )
            for data in iterations:
                if mode == "valid":
                    out, loss = self.forward_step(data, calc_loss=True)
                    self.valid_loss.update(loss.item(), n=data[0].size(0))
                elif mode == "test":
                    out, _ = self.forward_step(data, calc_loss=False)
                preds.append(out.detach().cpu().numpy())
        preds = np.concatenate(preds, axis=0)
        return preds

    def inference_loop_colwise(
        self,
        test_loader: DataLoader,
        mode: Literal["valid", "test"],
        best_score_dict: dict[str, tuple[int, float]],
    ):
        self.model.eval()
        if mode == "valid":
            self.valid_loss.reset()

        selected_counts = list(set([eval_count for eval_count, _ in best_score_dict.values()]))
        all_preds = np.zeros((len(test_loader.dataset), len(self.target_cols)))
        for eval_count in tqdm(selected_counts):
            self.model.load_state_dict(
                torch.load(
                    self.config.output_path / f"model{self.save_suffix}_eval{eval_count}.pth"
                )
            )
            preds = []
            with torch.no_grad():
                iterations = (
                    tqdm(test_loader, total=len(test_loader)) if self.detail_pbar else test_loader
                )
                for data in iterations:
                    if mode == "valid":
                        out, loss = self.forward_step(data, calc_loss=True)
                        self.valid_loss.update(loss.item(), n=data[0].size(0))
                    elif mode == "test":
                        out, _ = self.forward_step(data, calc_loss=False)
                    preds.append(out.detach().cpu().numpy())
            preds = np.concatenate(preds, axis=0)

            target_cols = [
                col for col, (count, _) in best_score_dict.items() if count == eval_count
            ]
            for col in target_cols:
                idx = self.target_cols.index(col)
                all_preds[:, idx] = preds[:, idx]
        return all_preds

    def update_best_score(self, indiv_scores: list[float], eval_count: int):
        update_num = 0
        for col, score in zip(self.target_cols, indiv_scores):
            if score > self.best_score_dict[col][1] and eval_count != -1:
                self.best_score_dict[col] = (eval_count, score)
                update_num += 1

        best_cw_score = (
            np.sum([score for _, score in self.best_score_dict.values()])
            + (368 - len(self.target_cols))
        ) / 368
        if update_num > 0 and eval_count != -1:
            pickle.dump(
                dict(self.best_score_dict),
                open(self.config.output_path / f"best_score_dict{self.save_suffix}.pkl", "wb"),
            )
        return best_cw_score, update_num

    def remove_unuse_weights(self):
        selected_counts = set([v[0] for v in self.best_score_dict.values()])
        weight_paths = list(self.config.output_path.glob(f"model{self.save_suffix}_eval*.pth"))
        for path in weight_paths:
            eval_count = int(path.stem.split("_")[-1].replace("eval", ""))
            if eval_count not in selected_counts:
                path.unlink()

    def forward_step(self, data: torch.Tensor, calc_loss: bool = True):
        if calc_loss:
            x, y = data
            x, y = x.to(self.config.device), y.to(self.config.device)
            out = self.model(x)
            loss = self.loss_fn(out, y)
        else:
            x = data[0]
            x = x.to(self.config.device)
            out = self.model(x)
            loss = None

        if self.config.target_shape == "3dim":
            out = self.convert_target_3dim_to_2dim(out)
        return out, loss

    def convert_target_3dim_to_2dim(
        self, y: np.ndarray | torch.Tensor
    ) -> np.ndarray | torch.Tensor:
        y_v = y[:, :, : len(VERTICAL_TARGET_COLS)]
        y_s = y[:, :, len(VERTICAL_TARGET_COLS) :]
        if isinstance(y, np.ndarray):
            y_v = np.transpose(y_v, (0, 2, 1)).reshape(y.shape[0], -1)
            y_s = y_s.mean(axis=1)
            y = np.concatenate([y_v, y_s], axis=-1)
        elif isinstance(y, torch.Tensor):
            y_v = y_v.permute(0, 2, 1).reshape(y.size(0), -1)
            y_s = y_s.mean(dim=1)
            y = torch.cat([y_v, y_s], dim=-1)
        y = self.alignment_target_idx(y)
        return y

    def alignment_target_idx(self, y: np.ndarray | torch.Tensor) -> np.ndarray | torch.Tensor:
        """
        target_colsとモデルの出力の順番を合わせる
        """
        align_order = [self.model_target_cols.index(col) for col in self.target_cols]
        assert len(y.shape) == 2
        y = y[:, align_order]
        return y

    def get_model_target_cols(self):
        model_target_cols = []
        for col in VERTICAL_TARGET_COLS:
            model_target_cols.extend([f"{col}_{i}" for i in range(60)])
        for col in SCALER_TARGET_COLS:
            model_target_cols.append(col)
        return model_target_cols

    def restore_pred(self, preds: np.ndarray):
        return preds * self.y_denominators + self.y_numerators

    def clipping_pred(self, preds: np.ndarray):
        for i in range(preds.shape[1]):
            preds[:, i] = np.clip(preds[:, i], self.target_min_max[i][0], self.target_min_max[i][1])
        return preds

    def save_oof_df(self, sample_ids: np.ndarray, preds: np.ndarray):
        oof_df = pl.DataFrame(preds, schema=self.target_cols)
        oof_df = oof_df.with_columns(sample_id=pl.Series(sample_ids))
        oof_df.write_parquet(self.config.oof_path / f"oof{self.save_suffix}.parquet")

    def postprocess(self, preds: np.ndarray, run_type: Literal["valid", "test"]):
        pp_x = self.valid_pp_df if run_type == "valid" else self.test_pp_df
        for x_col, y_col in zip(self.pp_x_cols, self.pp_y_cols):
            if y_col in self.target_cols:
                idx = self.target_cols.index(y_col)
                old_factor = self.old_factor_dict[y_col] if self.config.mul_old_factor else 1
                preds[:, idx] = (-1 * pp_x[x_col].to_numpy() / 1200) * old_factor
        return preds

    def load_postprocess_input(self, data_type: Literal["valid", "test"]):
        if data_type == "valid":
            valid_path = (
                self.config.input_path / "18_shrinked.parquet"
                if self.config.shared_valid
                else self.config.input_path / "train_shrinked.parquet"
            )
            self.valid_pp_df = (
                pl.scan_parquet(valid_path)
                .select(["sample_id"] + self.pp_x_cols)
                .filter(pl.col("sample_id").is_in(self.valid_ids))
                .collect()
            )
            id_df = pl.DataFrame({"sample_id": self.valid_ids})
            self.valid_pp_df = id_df.join(self.valid_pp_df, on="sample_id", how="left")

        elif data_type == "test":
            self.test_pp_df = pl.read_parquet(
                self.config.input_path / "test_shrinked.parquet",
                columns=["sample_id"] + self.pp_x_cols,
            )
            id_df = pl.DataFrame({"sample_id": self.test_ids})
            self.test_pp_df = id_df.join(self.test_pp_df, on="sample_id", how="left")


In [98]:
trainer = Trainer(config, logger)

In [99]:
oof_df = trainer.train(
    train_loader,
    valid_loader,
    colwise_mode=True,
)

27 28 0.6665454148778593


/tmp/ipykernel_32326/490323976.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(self.config.output_path / f"{retrain_weight_name}.pth")


  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/3375 [00:00<?, ?it/s]

  0%|          | 0/153 [00:00<?, ?it/s]

[ 2024-10-10 14:24:55 | INFO ] [Valid] : Epoch=0, Loss=0.16988, Score=0.66968, Best Col-Wise Score=0.67182


KeyboardInterrupt: 

In [91]:
oof_df[0]

0.6665454148778593

In [72]:
pred_df = trainer.test_predict(test_loader, eval_method="single")

/tmp/ipykernel_32326/383357159.py:246: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(self.config.output_path / f"model{self.save_suffix}_best.pth")


  0%|          | 0/153 [00:00<?, ?it/s]

In [71]:
pred_df

ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,ptend_t_36,…,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD,sample_id
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
0.072387,-1.447465,-1.59126,-1.238313,-1.039916,-0.886447,-0.759536,-0.794192,-0.957017,-1.032793,-0.944681,-0.791783,-0.54014,-0.306998,-0.069964,0.115484,0.786033,0.542847,0.249372,-0.374216,0.418973,-0.165187,-0.561241,-0.873283,-0.567651,-0.470924,-0.43505,-0.433015,-0.505128,-0.621606,-0.677111,-0.545636,-0.367299,-0.272195,-0.2401,-0.230634,-0.248424,…,0.256084,-0.121401,-0.344587,-0.102638,-0.091313,-0.140771,-0.460923,-0.049854,-0.018083,0.053758,-0.040552,0.309686,0.049827,0.035883,-0.286636,-0.126785,-1.287177,-2.573998,-2.5162,-1.333147,1.358848,1.754033,1.687201,0.85524,0.49837,0.349224,0.274072,0.017806,0.0,5.419843,0.0,0.06175,0.0,0.0,0.0,0.0,0
-0.485621,-1.135914,-0.649161,-0.742407,-0.981201,-1.199448,-1.358868,-1.201348,-0.899092,-0.662138,-0.59916,-0.602477,-0.670352,-0.698934,-0.682521,-0.733334,-0.659958,-0.355007,-0.285042,-0.208013,-0.244101,-0.429523,-0.397009,-0.343185,-0.364166,-0.398045,-0.450494,-0.475154,-0.492672,-0.480598,-0.430091,-0.338034,-0.181345,-0.009316,0.11043,0.261088,0.230349,…,-0.090072,0.020498,0.086971,0.050052,0.027946,0.063024,0.045506,0.026537,0.026066,-0.021002,-0.080584,-0.01897,0.002647,0.106812,-0.013483,-0.139997,-0.227484,-0.258315,-0.204068,-0.163616,-0.085005,-0.025923,-0.004916,0.093526,0.099184,0.026542,0.081104,0.141567,0.0,4.673466,0.0,0.039099,0.0,0.0,0.004282,0.007173,10
-0.234881,-1.656182,-0.601238,-0.433519,-0.789218,-0.849147,-0.75333,-0.671963,-0.595635,-0.537732,-0.55565,-0.608882,-0.711146,-0.758988,-0.750639,-0.721898,-0.592721,-0.432625,-0.277662,-0.208864,-0.280137,-0.387638,-0.296331,-0.302601,-0.214972,-0.178604,-0.198711,-0.220267,-0.217826,-0.22134,-0.288729,-0.324898,-0.344786,-0.334182,-0.341993,-0.344061,-0.341947,…,-0.214113,-0.028538,0.044159,0.040136,0.044838,0.076347,0.07342,0.041257,0.025995,0.00335,-0.010037,-0.010856,0.09887,0.040718,-0.088835,-0.134853,-0.455447,-0.750787,-1.185783,-1.05277,0.492555,0.40803,0.560029,0.34848,0.13647,0.193376,0.150923,0.374985,0.0,4.636291,0.035202,0.051106,0.0,0.0,0.0,0.0,100
0.841697,0.207503,0.660262,0.41642,-0.099689,-0.4974,-0.445526,-0.274708,-0.160325,-0.089188,-0.028821,-0.033144,-0.002816,0.130474,0.288755,0.572247,0.807359,0.651383,0.094121,0.031484,-0.032943,0.033807,-0.465906,-0.893544,-0.641397,-0.434837,-0.411204,-0.475317,-0.526809,-0.462754,-0.374046,-0.329558,-0.277853,-0.227232,-0.224293,-0.261834,-0.335193,…,-0.008861,0.15785,-0.316546,-0.006402,0.263694,-0.090587,0.003734,-0.151393,0.007677,-0.442158,1.166751,0.571663,0.326836,0.528868,0.470603,0.35881,0.213614,0.124506,-0.078239,-0.124431,-0.086841,-0.133892,-0.240608,-0.402967,-0.661201,-1.068136,-1.686045,0.885702,0.081611,4.937044,0.011568,0.000146,0.045463,0.068802,0.0976,0.022714,1000
1.500539,0.631652,0.84002,1.230621,1.381638,1.600752,1.813521,1.874617,1.767358,1.601102,1.472667,1.553341,1.733075,1.589896,1.570783,1.608648,1.6238

In [60]:
best_score, best_cw_score, best_epochs = trainer.train(
    train_loader,
    valid_loader,
    colwise_mode=True,
    eval_only=False
)

  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/3375 [00:00<?, ?it/s]

  0%|          | 0/153 [00:00<?, ?it/s]

[ 2024-10-10 10:07:30 | INFO ] [Valid] : Epoch=0, Loss=0.28119, Score=0.31449, Best Col-Wise Score=0.31449


  0%|          | 0/153 [00:00<?, ?it/s]

...

[ 2024-10-10 10:41:48 | INFO ] [Valid] : Epoch=0, Loss=0.17202, Score=0.66506, Best Col-Wise Score=0.66720


  0%|          | 0/153 [00:00<?, ?it/s]

[ 2024-10-10 10:43:19 | INFO ] [Valid] : Epoch=0, Loss=0.17192, Score=0.66655, Best Col-Wise Score=0.66900


  0%|          | 0/153 [00:00<?, ?it/s]

KeyboardInterrupt: 

# PostProcess

In [29]:
from omegaconf import DictConfig
import loguru
from typing import Literal
from sklearn.metrics import r2_score

from src.utils.competition_utils import get_sub_factor, get_io_columns

class PostProcess:
    def __init__(self, config: DictConfig, logger: loguru._Logger, additional: bool = True):
        self.config = config
        self.logger = logger
        self.additional = additional

        _, self.target_cols = get_io_columns(config)
        self.old_factor_dict = get_sub_factor(config.input_path, old=True)
        self.sub_cols = pl.read_parquet(config.input_path / 'sample_submission.parquet', n_rows=1).columns

        self.pp_x_cols = [f'state_q0002_{i}' for i in range(12, 27)]
        self.pp_y_cols = [f'ptend_q0002_{i}' for i in range(12, 27)]
        self.valid_pp_df = pl.read_parquet(
            config.input_path / '18_shrinked.parquet',
            columns=['sample_id'] + self.pp_x_cols
        )
        self.test_pp_df = pl.read_parquet(
            config.input_path / 'test_shrinked.parquet',
            columns=['sample_id'] + self.pp_x_cols
        )

        add_pp_y_cols = (
            [f'ptend_q0002_{i}' for i in range(60)] +
            [f'ptend_q0003_{i}' for i in range(60)]
        )
        self.add_pp_y_cols = [col for col in add_pp_y_cols if col in self.target_cols]
        self.add_pp_x_cols = [col.replace('ptend', 'state') for col in self.add_pp_y_cols]
        self.add_valid_pp_df = pl.read_parquet(
            config.input_path / '18_shrinked.parquet',
            columns=self.sub_cols + self.add_pp_x_cols
        )
        self.add_test_pp_df = pl.read_parquet(
            config.input_path / 'test_shrinked.parquet',
            columns=['sample_id'] + self.add_pp_x_cols
        )
        self.th_dict = None

    def postprocess(self, oof_df: pl.DataFrame, sub_df: pl.DataFrame):
        oof_df = self.complement_columns(oof_df)
        oof_df = self.reverse_sub_factor(oof_df)
        oof_df = self.replace_postprocess(oof_df, 'oof')

        sub_df = self.complement_columns(sub_df)
        sub_df = self.reverse_sub_factor(sub_df)
        sub_df = self.replace_postprocess(sub_df, 'sub')

        if self.additional:
            oof_df = self.additional_postprocess(oof_df, 'oof')
            sub_df = self.additional_postprocess(sub_df, 'sub')

        oof_df = self.create_oof_df(oof_df)
        sub_df = self.create_sub_df(sub_df)
        return oof_df, sub_df

    def complement_columns(self, pred_df: pl.DataFrame):
        lack_cols = list(set(self.sub_cols) - set(pred_df.columns))
        for col in lack_cols:
            pred_df = pred_df.with_columns([pl.lit(0).alias(col)])
        return pred_df

    def reverse_sub_factor(self, pred_df: pl.DataFrame):
        if self.config.mul_old_factor:
            exprs = []
            for col in self.target_cols:
                if self.old_factor_dict[col] != 0:
                    exprs.append((pl.col(col) / self.old_factor_dict[col]).alias(col))

            pred_df = pred_df.with_columns(exprs)
        return pred_df

    def replace_postprocess(self, pred_df: pl.DataFrame, pred_type: Literal['oof', 'sub']):
        pp_df = self.valid_pp_df if pred_type == 'oof' else self.test_pp_df
        pred_df = pred_df.join(pp_df, on=['sample_id'], how='left')

        exprs = []
        for x_col, y_col in zip(self.pp_x_cols, self.pp_y_cols):
            exprs.append((-1 * pl.col(x_col) / 1200).alias(y_col))
        pred_df = pred_df.with_columns(exprs)
        pred_df = pred_df.drop(self.pp_x_cols)
        return pred_df

    def additional_postprocess(self, pred_df: pl.DataFrame, pred_type: Literal['oof', 'sub']):
        pp_df = self.add_valid_pp_df if pred_type == 'oof' else self.add_test_pp_df
        pred_df = pred_df.join(pp_df, on=['sample_id'], how='left', suffix='_gt')
        exprs = []
        for x_col, y_col in zip(self.add_pp_x_cols, self.add_pp_y_cols):
            exprs.append((pl.col(x_col) + pl.col(y_col) * 1200).alias(f'{x_col}_next'))
        pred_df = pred_df.with_columns(exprs)

        if pred_type == 'oof':
            self.tuning_threshold(pred_df)

        assert self.th_dict is not None # oofから実行する必要がある
        exprs = []
        for y_col, (best_th, _) in self.th_dict.items():
            x_col = y_col.replace('ptend', 'state')
            exprs.append(
                pl.when(pl.col(f'{x_col}_next') < best_th)
                .then(-1 * pl.col(x_col) / 1200)
                .otherwise(pl.col(y_col))
                .alias(y_col)
            )
        pred_df = pred_df.with_columns(exprs)

        if pred_type == 'oof':
            scores = []
            for col in self.target_cols:
                score = r2_score(pred_df[f'{col}_gt'].to_numpy(), pred_df[col].to_numpy())
                scores.append(score)
            total_score = (np.sum(scores) + (368 - len(scores))) / 368
            self.logger.info(f'After Additional Postprocess: {total_score:.5f}')

        drop_cols = (
            self.add_pp_x_cols +
            [f'{col}_next' for col in self.add_pp_x_cols] +
            [col for col in pred_df.columns if '_gt' in col]
        )
        pred_df = pred_df.drop(drop_cols)
        return pred_df

    def tuning_threshold(self, pred_df: pl.DataFrame):
        iterations = tqdm(zip(self.add_pp_x_cols, self.add_pp_y_cols), total=len(self.add_pp_x_cols))
        for x_col, y_col in iterations:
            best_score = r2_score(pred_df[f'{y_col}_gt'].to_numpy(), pred_df[y_col].to_numpy())
            best_th = None
            for th_base in [0, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]:
                for corr in range(1, 10):
                    if th_base == 0 and corr >= 2:
                        break

                    th = th_base * corr
                    preds = pred_df.select(
                        pl.when(pl.col(f'{x_col}_next') < th)
                        .then(-1 * pl.col(x_col) / 1200)
                        .otherwise(pl.col(y_col))
                    ).to_numpy()

                    truths = pred_df[f'{y_col}_label'].to_numpy()
                    score = r2_score(truths, preds)
                    if score > best_score:
                        best_score = score
                        best_th = th

            if best_th is not None:
                self.th_dict[y_col] = (best_th, best_score)


    def create_oof_df(self, oof_df: pl.DataFrame):
        oof_df = oof_df.select(self.sub_cols)
        oof_df.write_parquet(self.config.oof_path / 'oof_pp.parquet')
        return oof_df

    def create_sub_df(self, sub_df: pl.DataFrame):
        sub_df = sub_df.with_columns(sample_id = pl.concat_str([pl.lit('test_'), pl.col('sample_id')]))
        sub_df = sub_df.select(self.sub_cols)
        sub_df.write_csv(self.config.output_path / 'submission_pp.csv')
        return sub_df